<a href="https://colab.research.google.com/github/ClaFlorez/Machine_Learning_Simplifie/blob/main/7_7_prediction_de_ventes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Projet complet de prédiction de ventes
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline # Import Pipeline

# Créer un dataset de ventes réaliste
print("PROJET: Système de prédiction de ventes pour chaîne de magasins")
print("=" * 80)

np.random.seed(42)
n_weeks = 104  # 2 ans de données hebdomadaires
n_stores = 50  # 50 magasins

# Générer les caractéristiques des magasins
store_data = []
for store_id in range(1, n_stores + 1):
    # Caractéristiques fixes du magasin
    surface = np.random.uniform(200, 1000)  # m²
    nb_employes = np.random.poisson(8) + 3  # 3-15 employés
    ville_taille = np.random.choice(['Petite', 'Moyenne', 'Grande'], p=[0.4, 0.4, 0.2])
    zone_type = np.random.choice(['Centre', 'Banlieue', 'Commercial'], p=[0.3, 0.5, 0.2])

    # Facteurs économiques locaux
    pouvoir_achat_local = np.random.normal(1.0, 0.2)  # Multiplicateur
    concurrence = np.random.poisson(3) + 1  # Nombre de concurrents

    store_data.append({
        'store_id': store_id,
        'surface': surface,
        'nb_employes': nb_employes,
        'ville_taille': ville_taille,
        'zone_type': zone_type,
        'pouvoir_achat_local': pouvoir_achat_local,
        'concurrence': concurrence
    })

stores_df = pd.DataFrame(store_data)

# Générer les données de ventes hebdomadaires
sales_data = []
base_date = datetime(2022, 1, 1)

for week in range(n_weeks):
    current_date = base_date + timedelta(weeks=week)

    # Facteurs temporels
    month = current_date.month
    is_holiday = month in [7, 8, 12]  # Vacances d'été et Noël
    is_winter = month in [12, 1, 2]
    week_of_year = current_date.isocalendar()[1]

    # Tendance générale (croissance de l'entreprise)
    trend_factor = 1 + (week * 0.002)  # 0.2% de croissance par semaine

    # Saisonnalité
    seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * week / 52)  # Cycle annuel

    for _, store in stores_df.iterrows():
        # Ventes de base selon les caractéristiques du magasin
        ventes_base = (
            store['surface'] * 50 +  # 50€ de CA par m²
            store['nb_employes'] * 2000 +  # 2000€ par employé
            store['pouvoir_achat_local'] * 10000 -  # Impact pouvoir d'achat
            store['concurrence'] * 1000  # Impact concurrence
        )

        # Ajustements selon la localisation
        if store['ville_taille'] == 'Grande':
            ventes_base *= 1.3
        elif store['ville_taille'] == 'Moyenne':
            ventes_base *= 1.1

        if store['zone_type'] == 'Centre':
            ventes_base *= 1.2
        elif store['zone_type'] == 'Commercial':
            ventes_base *= 1.4

        # Appliquer les facteurs temporels
        ventes_semaine = ventes_base * trend_factor * seasonal_factor

        # Effets spéciaux
        if is_holiday:
            ventes_semaine *= 1.5  # Boost pendant les vacances

        if is_winter and store['zone_type'] == 'Commercial':
            ventes_semaine *= 0.8  # Baisse hivernale pour centres commerciaux

        # Ajouter du bruit réaliste
        bruit = np.random.normal(0, ventes_semaine * 0.1)  # 10% de variabilité
        ventes_finale = max(0, ventes_semaine + bruit)  # Pas de ventes négatives

        sales_data.append({
            'store_id': store['store_id'],
            'date': current_date,
            'week_of_year': week_of_year,
            'month': month,
            'is_holiday': is_holiday,
            'is_winter': is_winter,
            'ventes': ventes_finale,
            'surface': store['surface'],
            'nb_employes': store['nb_employes'],
            'ville_taille': store['ville_taille'],
            'zone_type': store['zone_type'],
            'pouvoir_achat_local': store['pouvoir_achat_local'],
            'concurrence': store['concurrence']
        })

# Créer le DataFrame final
df = pd.DataFrame(sales_data)

print(f"Dataset de ventes créé:")
print(f"  Période: {df['date'].min().strftime('%Y-%m-%d')} à {df['date'].max().strftime('%Y-%m-%d')}")
print(f"  Nombre de magasins: {df['store_id'].nunique()}")
print(f"  Nombre de semaines: {df['week_of_year'].nunique()}")
print(f"  Total d'observations: {len(df):,}")
print(f"  Ventes moyennes: {df['ventes'].mean():,.0f}€/semaine")
print(f"  Écart-type: {df['ventes'].std():,.0f}€")

# Exploration des données
print(f"\nExploration des données:")
print("=" * 40)

# Ventes par type de magasin
print(f"Ventes moyennes par caractéristiques:")
for col in ['ville_taille', 'zone_type']:
    print(f"\n{col}:")
    avg_by_cat = df.groupby(col)['ventes'].mean().sort_values(ascending=False)
    for cat, avg in avg_by_cat.items():
        print(f"  {cat}: {avg:,.0f}€")

# Corrélations avec les ventes
numeric_cols = ['ventes', 'surface', 'nb_employes', 'pouvoir_achat_local', 'concurrence']
correlations = df[numeric_cols].corr()['ventes'].sort_values(ascending=False)

print(f"\nCorrélations avec les ventes:")
for col, corr in correlations.items():
    if col != 'ventes':
        print(f"  {col}: {corr:+.3f}")

# Préparer les features pour la régression
# Encoder les variables catégorielles
df_encoded = pd.get_dummies(df, columns=['ville_taille', 'zone_type'], prefix=['ville', 'zone'])

# Features pour la prédiction
feature_cols = [col for col in df_encoded.columns if col not in ['store_id', 'date', 'ventes']]
X = df_encoded[feature_cols]
y = df_encoded['ventes']

print(f"\nFeatures préparées:")
print(f"  Nombre de features: {len(feature_cols)}")
print(f"  Features: {feature_cols}")

# Diviser les données (attention: données temporelles!)
# On ne peut pas mélanger - les dernières semaines servent de test
split_point = int(len(df) * 0.8)  # 80% pour train, 20% pour test
df_sorted = df_encoded.sort_values(['store_id', 'date'])

X_train = df_sorted[feature_cols].iloc[:split_point]
X_test = df_sorted[feature_cols].iloc[split_point:]
y_train = df_sorted['ventes'].iloc[:split_point]
y_test = df_sorted['ventes'].iloc[split_point:]

print(f"\nDivision temporelle des données:")
print(f"  Entraînement: {len(X_train):,} observations")
print(f"  Test: {len(X_test):,} observations")
print(f"  Période test: dernières {len(X_test) // n_stores} semaines")

# Standardiser les features numériques
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Comparer plusieurs approches de régression
models = {
    'Linéaire Simple': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0, random_state=42),
    'Lasso (α=100)': Lasso(alpha=100, random_state=42, max_iter=2000),
    'Polynomiale Deg2': Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler()),
        ('ridge', Ridge(alpha=10, random_state=42))
    ])
}

print(f"\nComparaison des modèles de régression:")
print("=" * 80)
print(f"{'Modèle':<20} {'R² Train':<12} {'R² Test':<12} {'RMSE Test':<15} {'MAE Test'}")
print("-" * 80)

model_results = {}

for name, model in models.items():
    if 'Polynomiale' in name:
        # Pour le modèle polynomial, utiliser les données non standardisées
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        # Pour les autres, utiliser les données standardisées
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)

    # Calculer les métriques
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_test = mean_absolute_error(y_test, y_pred_test)

    model_results[name] = {
        'r2_train': r2_train,
        'r2_test': r2_test,
        'rmse_test': rmse_test,
        'mae_test': mae_test,
        'predictions': y_pred_test
    }

    print(f"{name:<20} {r2_train:<12.4f} {r2_test:<12.4f} {rmse_test:<15,.0f} {mae_test:,.0f}€")

# Identifier le meilleur modèle
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['r2_test'])
best_result = model_results[best_model_name]

print(f"\nMeilleur modèle: {best_model_name}")
print(f"  R² test: {best_result['r2_test']:.4f}")
print(f"  RMSE test: {best_result['rmse_test']:,.0f}€")
print(f"  MAE test: {best_result['mae_test']:,.0f}€")

# Analyser les prédictions par magasin
print(f"\nAnalyse des prédictions par magasin:")
print("=" * 50)

# Ajouter les prédictions au DataFrame de test
df_test = df_sorted.iloc[split_point:].copy()
df_test['predictions'] = best_result['predictions']
df_test['erreur_absolue'] = np.abs(df_test['ventes'] - df_test['predictions'])
df_test['erreur_relative'] = df_test['erreur_absolue'] / df_test['ventes'] * 100

# Performance par magasin
perf_by_store = df_test.groupby('store_id').agg({
    'ventes': 'mean',
    'predictions': 'mean',
    'erreur_absolue': 'mean',
    'erreur_relative': 'mean'
}).round(0)

# Top 5 meilleurs et pires magasins
best_stores = perf_by_store.nsmallest(5, 'erreur_relative')
worst_stores = perf_by_store.nlargest(5, 'erreur_relative')

print(f"Top 5 magasins avec meilleures prédictions:")
for store_id, row in best_stores.iterrows():
    print(f"  Magasin {store_id}: erreur moyenne {row['erreur_relative']:.1f}%")

print(f"\nTop 5 magasins avec prédictions les plus difficiles:")
for store_id, row in worst_stores.iterrows():
    print(f"  Magasin {store_id}: erreur moyenne {row['erreur_relative']:.1f}%")

# Analyse temporelle
print(f"\nAnalyse temporelle des prédictions:")
print("=" * 50)

# Performance par mois
df_test['month'] = df_test['date'].dt.month
perf_by_month = df_test.groupby('month').agg({
    'erreur_relative': 'mean',
    'ventes': 'mean'
}).round(1)

print(f"Performance par mois:")
mois_noms = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun',
             'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']

for month, row in perf_by_month.iterrows():
    nom_mois = mois_noms[month-1] if 1 <= month <= 12 else f"Mois {month}"
    print(f"  {nom_mois}: {row['erreur_relative']:.1f}% d'erreur moyenne")

# Impact business des erreurs
print(f"\nImpact business des erreurs de prédiction:")
print("=" * 60)

# Simuler les coûts
cout_surstockage = 0.02  # 2% du CA en coût de stockage par semaine
cout_rupture = 0.15  # 15% de perte de CA par rupture

surestimations = df_test['predictions'] > df_test['ventes']
sous_estimations = df_test['predictions'] < df_test['ventes']

# Calculs des coûts
erreurs_surestimation = df_test[surestimations]['predictions'] - df_test[surestimations]['ventes']
erreurs_sous_estimation = df_test[sous_estimations]['ventes'] - df_test[sous_estimations]['predictions']

cout_total_surstock = (erreurs_surestimation * cout_surstockage).sum()
cout_total_rupture = (erreurs_sous_estimation * cout_rupture).sum()
cout_total = cout_total_surstock + cout_total_rupture

print(f"Coûts estimés sur la période de test:")
print(f"  Surstockage: {cout_total_surstock:,.0f}€")
print(f"  Ruptures: {cout_total_rupture:,.0f}€")
print(f"  TOTAL: {cout_total:,.0f}€")

# Comparer avec une prédiction naïve (moyenne historique)
ventes_moyenne_historique = y_train.mean()
erreur_naive = np.abs(y_test - ventes_moyenne_historique)
cout_naive = (erreur_naive * (cout_surstockage + cout_rupture) / 2).sum()

economie = cout_naive - cout_total
print(f"\nComparaison avec prédiction naïve (moyenne):")
print(f"  Coût avec moyenne historique: {cout_naive:,.0f}€")
print(f"  Coût avec notre modèle: {cout_total:,.0f}€")
print(f"  ÉCONOMIE RÉALISÉE: {economie:,.0f}€")
print(f"  Amélioration: {economie/cout_naive*100:.1f}%")

# Recommandations opérationnelles
print(f"\nRecommandations opérationnelles:")
print("=" * 50)

if best_result['r2_test'] > 0.8:
    print("✅ EXCELLENT modèle - Déploiement recommandé")
    print("  • Utiliser pour les commandes automatiques")
    print("  • Monitorer les performances hebdomadairement")
elif best_result['r2_test'] > 0.6:
    print("✅ BON modèle - Déploiement avec supervision")
    print("  • Utiliser comme aide à la décision")
    print("  • Valider manuellement les commandes importantes")
else:
    print("⚠️ MODÈLE PERFECTIBLE - Amélioration nécessaire")
    print("  • Collecter plus de données")
    print("  • Ajouter des features (météo, événements locaux)")

# Stratégies d'amélioration
print(f"\nStratégies d'amélioration identifiées:")
print("=" * 50)

# Analyser les magasins difficiles à prédire
if len(worst_stores) > 0:
    print(f"Magasins difficiles à prédire:")
    for store_id, _ in worst_stores.iterrows():
        store_info = stores_df[stores_df['store_id'] == store_id].iloc[0]
        print(f"  Magasin {store_id}: {store_info['ville_taille']} ville, {store_info['zone_type']}")

    print(f"\nActions recommandées:")
    print("  • Analyser spécifiquement ces magasins")
    print("  • Collecter des données supplémentaires")
    print("  • Créer des modèles spécialisés si nécessaire")

# Analyser les périodes difficiles
mois_difficiles = perf_by_month[perf_by_month['erreur_relative'] > perf_by_month['erreur_relative'].mean() + perf_by_month['erreur_relative'].std()]

if len(mois_difficiles) > 0:
    print(f"\nPériodes difficiles à prédire:")
    for month, row in mois_difficiles.iterrows():
        nom_mois = mois_noms[month-1]
        print(f"  {nom_mois}: {row['erreur_relative']:.1f}% d'erreur")

    print(f"\nActions recommandées:")
    print("  • Ajouter des features saisonnières spécifiques")
    print("  • Intégrer les données météorologiques")
    print("  • Considérer les événements locaux")

# Système d'alertes
print(f"\nSystème d'alertes proposé:")
print("=" * 40)
print("ALERTE ROUGE (erreur >20%):")
print("  • Révision manuelle obligatoire")
print("  • Investigation des causes")

print("\nALERTE ORANGE (erreur 10-20%):")
print("  • Validation par le manager")
print("  • Ajustement possible")

print("\nVERT (erreur <10%):")
print("  • Commande automatique")
print("  • Monitoring standard")

# Métriques de succès pour le déploiement
print(f"\nMétriques de succès pour le suivi en production:")
print("=" * 60)
print("KPIs à surveiller:")
print(f"  • RMSE < {best_result['rmse_test']*1.1:,.0f}€ (tolérance +10%)")
print(f"  • MAE < {best_result['mae_test']*1.1:,.0f}€")
print(f"  • R² > {best_result['r2_test']*0.9:.3f} (tolérance -10%)")
print(f"  • Erreur relative moyenne < 15%")

print(f"\nFréquence de réentraînement:")
print("  • Réentraînement mensuel recommandé")
print("  • Réentraînement d'urgence si performance dégradée")
print("  • Intégration de nouvelles features trimestrielle")

# ROI du projet
print(f"\nROI estimé du projet:")
print("=" * 30)
economie_annuelle = economie * (52 / (len(X_test) // n_stores))  # Extrapoler sur 1 an
cout_developpement = 50000  # Estimation
roi = (economie_annuelle - cout_developpement) / cout_developpement * 100

print(f"  Économie annuelle estimée: {economie_annuelle:,.0f}€")
print(f"  Coût de développement: {cout_developpement:,.0f}€")
print(f"  ROI première année: {roi:.0f}%")

if roi > 100:
    print("  ✅ Projet très rentable!")
elif roi > 50:
    print("  ✅ Projet rentable")
elif roi > 0:
    print("  ⚠️ Projet marginalement rentable")
else:
    print("  ❌ Projet non rentable en l'état")

PROJET: Système de prédiction de ventes pour chaîne de magasins
Dataset de ventes créé:
  Période: 2022-01-01 à 2023-12-23
  Nombre de magasins: 50
  Nombre de semaines: 52
  Total d'observations: 5,200
  Ventes moyennes: 87,441€/semaine
  Écart-type: 35,419€

Exploration des données:
Ventes moyennes par caractéristiques:

ville_taille:
  Grande: 111,506€
  Moyenne: 94,515€
  Petite: 73,407€

zone_type:
  Centre: 102,643€
  Commercial: 91,979€
  Banlieue: 77,921€

Corrélations avec les ventes:
  surface: +0.634
  nb_employes: +0.347
  pouvoir_achat_local: +0.142
  concurrence: -0.124

Features préparées:
  Nombre de features: 14
  Features: ['week_of_year', 'month', 'is_holiday', 'is_winter', 'surface', 'nb_employes', 'pouvoir_achat_local', 'concurrence', 'ville_Grande', 'ville_Moyenne', 'ville_Petite', 'zone_Banlieue', 'zone_Centre', 'zone_Commercial']

Division temporelle des données:
  Entraînement: 4,160 observations
  Test: 1,040 observations
  Période test: dernières 20 semaines
